# Simulación 9

## Simlib

### 1. Definir:

**a. Parámetros de entrada**

Tal como se define en el enunciado del problema, la simulación debe recibir tres parámetros, los que definen los tiempos de la simulación.
A estos tres, añadimos un cuarto parámetro, que determina si los tiempos van a ser exponenciales, o constantes. Esto es con el fin de usar el mismo
archivo de código para cada una de las corridas, en vez de crear archivos separados. Los parámetros servers_level1 y servers_level2 indican el número de servidores de cada servicio. En los primeros cuatro puntos, se mantendrá constante de 1 servidor por cada servicio, pero analizaremos el resultado de duplicar servidores en el punto 5.

| Nombre                | Parámetro                                                   |
|-----------------------|-------------------------------------------------------------|
| mean_interarrival     | Tiempo promedio de llegada entre elementos                  |
| mean_service          | Tiempo promedio de servicio en cada uno de los dos procesos |
| total_simulation_time | Tiempo total de la simulación                               |
| distribution_type     | Tipo de distribuciones usadas en la simulación              |
| servers_level1        | Número de servidores del servicio 1                         |
| servers_level2        | Número de servidores del servicio 2                         |

La variable distribution_type, puede tomar valores de 1 al 4, para las distintas corridas del programa:

| Valor | mean_interarrival | mean_service |
|-------|-------------------|--------------|
| 1     | Exponencial       | Exponencial  |
| 2     | Constante         | Exponencial  |
| 3     | Exponencial       | Constante    |
| 4     | Constante         | Constante    |

**b. Variables del modelado**

El uso de Simlib, permite facilitar la declaración de variables explícitas, y la definición de los métodos de la simulación. Esto es, porque la propia librería tiene predefinidos múltiples variables y métodos en cada simulación, realizada con esta. Por tanto, muchas de las variables específicas de cada agente, evento o proceso, son manejadas internamente por Simlib, y no tienen que ser escritas en el código. En la siguiente tabla, mostramos las variables usadas de forma explícita en la simulación.

| Nombre          | Tipo  | Uso                                                                 |
|-----------------|-------|---------------------------------------------------------------------|
| sim_time        | Float | Reloj interno del simulador                                         |
| system_time     | Float | Tiempo del elemento en el sistema                                   |
| next_event_type | Int   | Identificador del tipo de evento que se debe planear a continuación |
| maxatr          | Int   | Número máximo de atributos por lista                                |
| maxlist         | Int   | Número máximo de listas                                             |

Mantenemos las llamadas al reloj interno, y al identificador de tipo de evento. El reloj sigue siendo necesario manejarlo, para especificar el manejo de eventos de esta simulación en particular. Por ejemplo, en este programa que requiere indicar de entrada, el tipo de distribuciones (exponencial o constante) con el que se van a programar las llegadas y servicios de los elementos, es necesario especificar cómo se van a calcular, usando sim_time como referente. Creamos una nueva variable "system_time", para realizar seguimiento del tiempo de cada elemento dentro del sistema. Esto es porque, el uso del arreglo transfer es susceptible a que se sobreesciba su contenido, cada vez que se realiza una nueva operación. Por tanto, necesitamos guardar los contenidos necesarios en esta variable, si se quiere hacer el seguimiento del elemento en todos los eventos. El identificador de tipo de evento es necesario para dar forma a la simulación particular. Por ejemplo, es necesario identificar el orden para crear una simulación de dos servicios secuenciales, en vez de paralelos.
Finalmente, maxatr y maxlist deben ser definidos en cualquier simulación que utilice simlib. La variable maxatr define el número máximo de atributos de cada lista simlib. Por la manera en la que esta escrito Simlib internamente, el mínimo valor posible de maxatr es 4. Por otra parte, maxlist indica el máximo de listas que se pueden crear en una simulación. El mínimo valor que se puede usar es 25, porque la lista 25 siempre va a ser la lista de eventos.


**c. Descripción del evento y tipo de evento**

| Nombre     | Tipo | Uso                                                                                                           |
|------------|------|---------------------------------------------------------------------------------------------------------------|
| ARRIVAL    | 1    | Llegada de un elemento a la cadena de producción, puede ser atendido o hacer fila si el servidor está ocupado |
| DEPARTURE1 | 2    | Terminación del primer servicio. Apenas de termina el Servicio 1, es atendido, o hace fila para el Servicio 2 |
| DEPARTURE2 | 3    | Terminación del segundo servicio. Cuando termina el Servicio 2, el elemento sale del sistema                  |

La simulación 9, en sus cuatro variantes, es muy parecida al ejercicio de prueba del Capítulo 1 del libro de Averill y Law. La principal diferencia, es que los elementos deben recibir dos servicios, en vez de uno, y para recibir el servicio 2, debe haber terminado el servicio 1. Por esta razón, se definen dos servicios de salida, uno para cada servicio. Sus diferencias, es la manera en la que se maneja el elemento respecto al servicio. Los elementos sólo salen del sistema si se termina el servicio 2. En cambio, la finalización del servicio 1 indica que debe iniciar el servicio 2, el cual tiene una cola infinita, y sólo debe trasladar el elemento a la segunda cola. La cola de llegada al sistema es específica para recibir el servicio 1.



**d. Listas y sus atributos**

| Número | Nombre     | Atributo 1                    | Atributo 2     |                                                                              
|--------|------------|-------------------------------|----------------|
| 1      | QUEUE1     | Tiempo de llegada a la fila 1 |                |
| 2      | SERVER1    |                               |                |
| 3      | QUEUE2     | Tiempo de llegada a la fila 2 |                |
| 4      | SERVER2    |                               |                |
| 25     | EVENT_LIST | Tiempo de inicio del evento   | Tipo de evento |
 
Como ya se comentó en el punto anterior, la lista 25 existe en todas las simulaciones de Simlib, y siempre es por defecto la lista de eventos. Por tanto, esta lista guarda dos atibutos por cada evento, el tiempo en el que se registra el evento, y el tipo de evento registrado. Para las otras listas, los argumentos se manejan de forma diferente. Para las listas 1 y 3 (filas), se registra únicamente el tiempo de llegada a la fila. El método arrive registra el tiempo de reloj en QUEUE1 cuando se procesa un evento de llegada, y el servidor 1 está disponible, y el método depart1 hace lo mismo con el evento de salida 1, y el servidor 2 disponible. Por otra parte, las listas 2 y 4 (servidores), únicamente registran un booleano, para indicar si está disponible u ocupado el servidor. Esto lo verifican los métodos arrive y depart1, al consultar si el valor es igual a 1, están preguntando si el servidor 1 y 2 respectivamente están disponibles.

**e. Contadores y/o acumuladores**

| Variable                | Tipo            | Descripción                                           | 
|-------------------------|-----------------|-------------------------------------------------------|
| num_custs_delayed       | Contador entero | Clientes que no han sido atendidos de forma inmediata |
| num_completed_customers | Contador entero | Clientes que han completado el servicio 2             |

Como se comentó en los puntos anteriores, la mayoría de variables y contadores son manejadas internamente por la librería Simlib, al realizar los métodos predefinidos de Simlib. En la simulación sólo se manejan dos contadores, que permiten contar la cantidad de clientes que han sido atendidos.

**f. Medidas de desempeño**

| Variable                 | Descripción                                           | 
|--------------------------|-----------------------------------------------------|
| Demora promedio en cola       | Tiempo promedio de espera en cualquiera de las dos colas            |
| Demora máxima en cola         | Tiempo máximo de espera registrado en cola |
| Demora mínima en cola         | Tiempo mínimo de espera registrado en cola |
| Longitud promedio de cola 1   | Contador entero | Clientes que no han sido atendidos de forma inmediata |
| Longitud máxima de cola 1     | Contador entero | Clientes que no han sido atendidos de forma inmediata |
| Longitud mínima de cola 1     | Contador entero | Clientes que no han sido atendidos de forma inmediata |
| Longitud promedio de cola 2   | Contador entero | Clientes que no han sido atendidos de forma inmediata |
| Longitud máxima de cola 2     | Contador entero | Clientes que no han sido atendidos de forma inmediata |
| Longitud mínima de cola 2     | Contador entero | Clientes que no han sido atendidos de forma inmediata |
| Uso promedio del servidor 1   | Contador entero | Clientes que no han sido atendidos de forma inmediata |
| Uso máximo del servidor 1     | Contador entero | Clientes que no han sido atendidos de forma inmediata |
| Uso mínimo del servidor 1     | Contador entero | Clientes que no han sido atendidos de forma inmediata |
| Uso promedio del servidor 2   | Contador entero | Clientes que no han sido atendidos de forma inmediata |
| Uso máximo del servidor 2     | Contador entero | Clientes que no han sido atendidos de forma inmediata |
| Uso mínimo del servidor 2     | Contador entero | Clientes que no han sido atendidos de forma inmediata |

**g. Subprogramas y propósito**

En la solución presentada, no se crearon subprogramas para las simulaciones. Como tal, era posible crear cuatro archivos separados, especificando las distribuciones de cada corrida. No obstante, en el simulador utilizado para los resultados, se prefirió usar un solo programa, el cual pudiera cambiar las distribuciones entre exponencial y constante, dependiendo del modo que se le suministra.

### 2. Elaborar el diagrama de flujo del programa principal y de cada subprograma que conforma el modelo

### 3. Desarrollar el simulador en lenguaje de alto nivel

In [8]:
%%writefile parameters_9.txt
10 9 10000 1 1 1

Writing parameters_10.txt


%%writefile simulation_9.c
#include "simlib.h"

#define EVENT_ARRIVAL 1
#define EVENT_DEPARTURE1 2
#define EVENT_DEPARTURE2 3
#define LIST_QUEUE1 1
#define LIST_SERVER1 2
#define LIST_QUEUE2 3
#define LIST_SERVER2 4
#define SAMPST_DELAYS 1
#define STREAM_INTERARRIVAL 1
#define STREAM_SERVICE 2

int num_custs_delayed, total_simulation_time, num_completed_customers, 
distribution_type,servers_level1, servers_level2;
float mean_interarrival, mean_service;
FILE *infile, *outfile;

float get_interarrival_time() {
    switch (distribution_type) {
        case 1: // Exponential interarrival
        case 3:
            return expon(mean_interarrival, STREAM_INTERARRIVAL);
        case 2: // Constant interarrival
        case 4:
            return mean_interarrival; // Constant time = mean
        default:
            return expon(mean_interarrival, STREAM_INTERARRIVAL); // Default to exponential
    }
}

float get_service_time() {
    switch (distribution_type) {
        case 1: // Exponential service
        case 2:
            return expon(mean_service, STREAM_SERVICE);
        case 3: // Constant service
        case 4:
            return mean_service; // Constant time = mean
        default:
            return expon(mean_service, STREAM_SERVICE); // Default to exponential
    }
}

void init_model(void) {
    num_custs_delayed = 0;
    event_schedule(sim_time + get_interarrival_time(), EVENT_ARRIVAL);
}

void arrive(void) {
    event_schedule(sim_time + get_interarrival_time(), EVENT_ARRIVAL);
    
    if (list_size[LIST_SERVER1] >= servers_level1) {  // Check against number of servers
        transfer[1] = sim_time;
        list_file(LAST, LIST_QUEUE1);
    }
    else {
        sampst(0.0, SAMPST_DELAYS);
        ++num_custs_delayed;
        list_file(FIRST, LIST_SERVER1);
        event_schedule(sim_time + get_service_time(), EVENT_DEPARTURE1);
    }
}

void depart1(void) {
    if (list_size[LIST_QUEUE1] == 0) {
        list_remove(FIRST, LIST_SERVER1);
    } else {
        list_remove(FIRST, LIST_QUEUE1);
        sampst(sim_time - transfer[1], SAMPST_DELAYS);
        ++num_custs_delayed;
        event_schedule(sim_time + get_service_time(), EVENT_DEPARTURE1);
    }

    if (list_size[LIST_SERVER2] >= servers_level2) {  // Check against number of servers
        transfer[1] = sim_time;
        list_file(LAST, LIST_QUEUE2);
    }
    else {
        list_file(FIRST, LIST_SERVER2);
        event_schedule(sim_time + get_service_time(), EVENT_DEPARTURE2);
    }
}

void depart2(void) {
    if (list_size[LIST_QUEUE2] == 0) {
        list_remove(FIRST, LIST_SERVER2);
    } else {
        list_remove(FIRST, LIST_QUEUE2);
        event_schedule(sim_time + get_service_time(), EVENT_DEPARTURE2);
    }

    ++num_completed_customers;
}

void report(void) {
    fprintf(outfile, "\nDelays in queue, in minutes:\n");
    out_sampst(outfile, SAMPST_DELAYS, SAMPST_DELAYS);
    fprintf(outfile, "\nQueue length and server utilization:\n");
    out_filest(outfile, LIST_QUEUE1, LIST_SERVER1);
    out_filest(outfile, LIST_QUEUE2, LIST_SERVER2);
    fprintf(outfile, "\nTime simulation ended:%12.3f minutes\n", sim_time);
    fprintf(outfile, "Number of customers completed both services:%8d\n", num_completed_customers);
    fprintf(outfile, "Number of servers at level 1: %d\n", servers_level1);
    fprintf(outfile, "Number of servers at level 2: %d\n", servers_level2);
}

int main() {
    char outfilename[20];

    infile = fopen("parameters_9.txt", "r");
    if (infile == NULL) {
        printf("Error: cannot open input file parameters_9.txt\n");
        return 1;
    }

    // Read input parameters including server counts
    fscanf(infile, "%f %f %d %d %d %d", 
           &mean_interarrival, &mean_service, 
           &total_simulation_time, &distribution_type,
           &servers_level1, &servers_level2);

    // Generate output filename
    sprintf(outfilename, "report_9_%d.txt", distribution_type, servers_level1, servers_level2);

    outfile = fopen(outfilename, "w");
    if (outfile == NULL) {
        printf("Error: cannot open output file %s\n", outfilename);
        fclose(infile);
        return 1;
    }

    fprintf(outfile, "Multi-server queueing system using simlib\n\n");
    fprintf(outfile, "Mean interarrival time%11.3f minutes\n\n", mean_interarrival);
    fprintf(outfile, "Mean service time%16.3f minutes\n\n", mean_service);
    fprintf(outfile, "Total simulation time%12d minutes\n\n", total_simulation_time);
    fprintf(outfile, "Distribution type: %d\n", distribution_type);
    switch (distribution_type) {
        case 1:
            fprintf(outfile, "Interarrival: Exponential, Service: Exponential\n");
            break;
        case 2:
            fprintf(outfile, "Interarrival: Constant, Service: Exponential\n");
            break;
        case 3:
            fprintf(outfile, "Interarrival: Exponential, Service: Constant\n");
            break;
        case 4:
            fprintf(outfile, "Interarrival: Constant, Service: Constant\n");
            break;
        default:
            fprintf(outfile, "Invalid distribution type. Defaulting to Exponential/Exponential.\n");
    }
    fprintf(outfile, "Number of servers at level 1: %d\n", servers_level1);
    fprintf(outfile, "Number of servers at level 2: %d\n\n", servers_level2);

    maxatr = 4;
    maxlist = 26;
    init_simlib();
    init_model();
    while (sim_time < total_simulation_time) {
        timing();
        switch (next_event_type) {
            case EVENT_ARRIVAL:
                arrive();
                break;
            case EVENT_DEPARTURE1:
                depart1();
                break;
            case EVENT_DEPARTURE2:
                depart2();
                break;
        }
    }
    report();
    fclose(infile);
    fclose(outfile);

    return 0;
}

### 4. Analizar resultados

In [10]:
!gcc simulation_9.c simlib.c -o simulation_9 -lm

In file included from simlib.h:9,
                 from simulation_10.c:74:
simlibdefs.h:30: warning: "INFINITY" redefined
   30 | #define INFINITY     1.E30  /* Not really infinity, but a very large number. */
      | 
In file included from simulation_10.c:73:
/usr/include/math.h:91: note: this is the location of the previous definition
   91 | #  define INFINITY (__builtin_inff ())
      | 
In file included from simlib.c:9:
simlibdefs.h:30: warning: "INFINITY" redefined
   30 | #define INFINITY     1.E30  /* Not really infinity, but a very large number. */
      | 
In file included from simlib.c:8:
/usr/include/math.h:91: note: this is the location of the previous definition
   91 | #  define INFINITY (__builtin_inff ())
      | 


In [11]:
!./simulation_9

In [12]:
with open("report_9_1.txt", "r") as file:
    print(file.read())


FileNotFoundError: [Errno 2] No such file or directory: '/content/report_10.txt'

**a.	Corrida 1: tiempo entre llegada y de servicio exponenciales**

Multi-server queueing system using simlib

Mean interarrival time     10.000 minutes

Mean service time           9.000 minutes

Total simulation time       10000 minutes

Distribution type: 1
Interarrival: Exponential, Service: Exponential
Number of servers at level 1: 1
Number of servers at level 2: 1


Delays in queue, in minutes:

 sampst                         Number
variable                          of
 number       Average           values          Maximum          Minimum
________________________________________________________________________

    1         121.123          1014.00          349.119          0.00000 
________________________________________________________________________



Queue length and server utilization:

  File         Time
 number       average          Maximum          Minimum
_______________________________________________________

    1         12.2813          37.0000          0.00000 

    2        0.925342          1.00000          0.00000 
_______________________________________________________



  File         Time
 number       average          Maximum          Minimum
_______________________________________________________

    3         5.06968          28.0000          0.00000 

    4        0.862937          1.00000          0.00000 
_______________________________________________________



Time simulation ended:   10000.495 minutes
Number of customers completed both services:    1006
Number of servers at level 1: 1
Number of servers at level 2: 1


**b.	Corrida 2: Tiempo entre llegada constantes y tiempos de servicio exponenciales**

In [8]:
%%writefile parameters_9.txt
10 9 10000 2 1 1

Writing parameters_10.txt


In [11]:
!./simulation_9

In [12]:
with open("report_9_2.txt", "r") as file:
    print(file.read())


FileNotFoundError: [Errno 2] No such file or directory: '/content/report_10.txt'

Multi-server queueing system using simlib

Mean interarrival time     10.000 minutes

Mean service time           9.000 minutes

Total simulation time       10000 minutes

Distribution type: 2
Interarrival: Constant, Service: Exponential
Number of servers at level 1: 1
Number of servers at level 2: 1


Delays in queue, in minutes:

 sampst                         Number
variable                          of
 number       Average           values          Maximum          Minimum
________________________________________________________________________

    1         36.6710          998.000          148.308          0.00000 
________________________________________________________________________



Queue length and server utilization:

  File         Time
 number       average          Maximum          Minimum
_______________________________________________________

    1         3.66076          15.0000          0.00000 

    2        0.894527          1.00000          0.00000 
_______________________________________________________



  File         Time
 number       average          Maximum          Minimum
_______________________________________________________

    3         3.54243          25.0000          0.00000 

    4        0.864306          1.00000          0.00000 
_______________________________________________________



Time simulation ended:   10000.000 minutes
Number of customers completed both services:     993
Number of servers at level 1: 1
Number of servers at level 2: 1


**c.	Corrida 3: Tiempo entre llegadas exponenciales y tiempo de servicio constantes**

In [8]:
%%writefile parameters_9.txt
10 9 10000 3 1 1

Writing parameters_10.txt


In [11]:
!./simulation_9

In [12]:
with open("report_9_3.txt", "r") as file:
    print(file.read())


FileNotFoundError: [Errno 2] No such file or directory: '/content/report_10.txt'

Multi-server queueing system using simlib

Mean interarrival time     10.000 minutes

Mean service time           9.000 minutes

Total simulation time       10000 minutes

Distribution type: 3
Interarrival: Exponential, Service: Constant
Number of servers at level 1: 1
Number of servers at level 2: 1


Delays in queue, in minutes:

 sampst                         Number
variable                          of
 number       Average           values          Maximum          Minimum
________________________________________________________________________

    1         47.3141          1015.00          208.489          0.00000 
________________________________________________________________________



Queue length and server utilization:

  File         Time
 number       average          Maximum          Minimum
_______________________________________________________

    1         4.79695          24.0000          0.00000 

    2        0.911567          1.00000          0.00000 
_______________________________________________________



  File         Time
 number       average          Maximum          Minimum
_______________________________________________________

    3         0.00000          1.00000          0.00000 

    4        0.911567          1.00000          0.00000 
_______________________________________________________



Time simulation ended:   10011.330 minutes
Number of customers completed both services:    1014
Number of servers at level 1: 1
Number of servers at level 2: 1


**d.	Corrida 4: Tiempo entre llegadas constantes y tiempos de servicio constantes**

In [8]:
%%writefile parameters_9.txt
10 9 10000 4 1 1

Writing parameters_10.txt


In [11]:
!./simulation_9

In [12]:
with open("report_9_4.txt", "r") as file:
    print(file.read())


FileNotFoundError: [Errno 2] No such file or directory: '/content/report_10.txt'

Multi-server queueing system using simlib

Mean interarrival time     10.000 minutes

Mean service time           9.000 minutes

Total simulation time       10000 minutes

Distribution type: 4
Interarrival: Constant, Service: Constant
Number of servers at level 1: 1
Number of servers at level 2: 1


Delays in queue, in minutes:

 sampst                         Number
variable                          of
 number       Average           values          Maximum          Minimum
________________________________________________________________________

    1         0.00000          1000.00          0.00000          0.00000 
________________________________________________________________________



Queue length and server utilization:

  File         Time
 number       average          Maximum          Minimum
_______________________________________________________

    1         0.00000     -1.00000E+30      1.00000E+30 

    2        0.899100          1.00000          0.00000 
_______________________________________________________



  File         Time
 number       average          Maximum          Minimum
_______________________________________________________

    3         0.00000     -1.00000E+30      1.00000E+30 

    4        0.898300          1.00000          0.00000 
_______________________________________________________



Time simulation ended:   10000.000 minutes
Number of customers completed both services:     998
Number of servers at level 1: 1
Number of servers at level 2: 1


**e. Análisis de resultados**

En esta sección, acabamos de mostrar cómo se realizan reportes de rendimiento para las cuatro corridas requeridas para el ejercicio. En la siguiente tabla, comparamos las corridas realizadas durante el desarrollo de este taller. Se pueden confirmar estos valores en el repositorio oficial del taller:
| Métrica            | Exponencial/Exponencial | Constante/Exponencial | Exponencial/Constante | Constante/Constante |
|--------------------|-------------------------|-----------------------|-----------------------|---------------------|
| Demora total       | 197.448 (0.9-603.5)     | 124.112 (3.3-286.1)   | 65.3607 (18-226.5)    | 18 (18-18)          |
| Demora cola 1      | 70.3611 (0-270.2)       | 34.0812 (0-212.6)     | 47.3141 (0-208.5)     | 0                   |
| Demora cola 2      | 109.225 (0-520.8)       | 72.1279 (0-235.2)     | 0                     | 0                   |
| Longitud fila 1    | 7.13311 (0-26)          | 3.40471 (0-22)        | 4.79695 (0-24)        | 0                   |
| Longitud fila 2    | 11.0731 (0-57)          | 7.17686 (0-25)        | 0                     | 0                   |
| Uso servidor 1     | 90.6411%                | 84.8335%              | 91.1567%              | 89.91%              |
| Uso servidor 2     | 88.6811%                | 91.0061%              | 91.1567%              | 89.83%              |
| Clientes atendidos | 1013                    | 992                   | 1014                  | 998                 |

Primero, debemos confirmar que estos valores sean coherentes con las características de cada corrida. Se observa que, en general, la simulación con tiempos de llegada y servicio constante, es la más eficiente, ya que evita la formación de colas, y la demora total promedio de los clientes es la más baja de las cuatro corridas (18). Por otra parte, la primera corrida, la de tiempos exponenciales, es la que tiene mayor tiempo de demora en atención al cliente, así como las demoras más largas en la cola, y los peores casos más exagerados. 
El resultado de la corrida 4 tan exacto es coherente. El tiempo de demora total no considera el tiempo entre llegadas de cada cliente. Por tanto, el tiempo de demora total es, el tiempo de ambos servicios, más el tiempo de espera en ambas colas. Si el tiempo de servicio es constante (9 minutos), y ambos servicios son iguales, un tiempo promedio de demora de 18 minutos indica que no hubo demora en cola, que es exactamente lo que se observa en las demoras de cada cola. Tiene sentido que no se genere nunca cola en esta simulación, porque el tiempo entre llegadas es constante, y mayor al tiempo de servicio, por lo que siempre se libera el servidor antes de que llegue un nuevo cliente.
Por otra parte, la diferencia entre los casos exponenciales con los constantes también es coherente. Aunque el promedio de llegada, como el de servicio, son iguales en ambas distribuciones, la variabilidad en la distribución exponencial es mucho mayor. Esto implica, que van a existir casos de demoras en servicio, así como tiempos de llegada inusualmente rápidos La presencia de casos anómalos a lo largo de la simulación van a generar retrasos, que se manifiestan como el crecimiento de las colas. Incluso en el peor caso (Corrida 1), el promedio de longitud de fila no es tan grande (7 y 11 respectivamente), pero los máximos registrados muestran que existen momentos específicos en los que las colas crecen significativamente, y por tanto, los tiempos de espera resultantes. Cualquier situación de tiempo de llegada menor a alguno de los dos tiempos de servicio de los dos servidores generará un retraso, que se va a ir acumulando en los clientes posteriores.
Comparando la corrida 2 con la corrida 3, observamos que hacer constante el tiempo de servicio, tiene mejores resultados que hacer constante el tiempo de espera. Esto también tiene sentido, ya que el tiempo de servicio tiene un mayor impacto en la simulación general, al presentar dos servidores de manera secuencial. No solo es (en el caso óptimo), más demorado atender un cliente en ambos servidores, que esperar a un nuevo cliente, sino que además, una demora en cualquiera de los dos servidores va a generar un retraso en el servicio. Por eso se observa que la Corrida 3 no tiene fila para el servicio 2. Como los tiempos son constantes, la llegada de clientes al servicio 2 depende exclusivamente de la llegada y resolución de la cola al servicio 1. Una vez se inicia el servicio 1, el tiempo de servicio total es constante, y será más rápido que el proceso de desencolar al servicio 1.
Finalmente, llama la atención que, aunque el tiempo de espera por cliente difiere significativamente en cada corrida, el número de clientes atendidos se mantiene constante. Esto se explica por el porcentaje de uso del servidor. Aunque el tiempo de espera del cliente sea distinto, el tiempo de atención promedio se mantiene constante. Esto se observa porque en las cuatro corridas, el uso de los servidores es casi constante. Lo que espera observarse, es que la fila pendiente al final de la simulación de la corrida 1, sea mayor que la fila de las corridas 3 y 4 (que nunca se encola), pero el tiempo que demora cada servidor en atender a los clientes, se mantiene muy constante. 

### 5. Plantear modificaciones

Para el proceso de modificaciones, vamos a realizar experimentos únicamente en parámetros que se presume, pueden ser modificados por el sistema. Por ejemplo, el tiempo de llegada de clientes (y su distribución), no es dependiente del funcionamiento de su sistema, y se presume que no puede ser modificado por acciones reales del sistema. Por tanto, nos vamos a enfocar en dos variables, el número de servidores, y el tiempo promedio de servicio. Comparar estas dos variables, nos permite asesorar, por ejemplo, si es más eficiente poner una nueva máquina (en el caso de que sea una planta de ensamblaje, equivalente a duplicar servidores), o si es más eficiente reemplazar la máquina existente por una más rápida (disminuyendo el tiempo de servicio). También es importante determinar cuál de las dos máquinas vale más la pena duplicar o reemplazar. A continuación, mostramos las corridas para el modo 1, con las modificaciones pertinentes:

## Simpy

### 1. Definir:

**a. Parámetros de entrada**


| Nombre                | Parámetro                                                   |
|-----------------------|-------------------------------------------------------------|
| mean_interarrival     | Tiempo promedio de llegada entre elementos                  |
| mean_service          | Tiempo promedio de servicio en cada uno de los dos procesos |
| total_simulation_time | Tiempo total de la simulación                               |
| distribution_type     | Tipo de distribuciones usadas en la simulación              |
| servers_level1        | Número de servidores del servicio 1                         |
| servers_level2        | Número de servidores del servicio 2                         |


| Valor | mean_interarrival | mean_service |
|-------|-------------------|--------------|
| 1     | Exponencial       | Exponencial  |
| 2     | Constante         | Exponencial  |
| 3     | Exponencial       | Constante    |
| 4     | Constante         | Constante    |

Los parámetros que usa el programa de entrada en Simpy son los mismos que se usaron para la simulación en Simlib. Nótese que la variable total_simulation_type no es declarada en el constructor de la clase, porque no es usada por la clase. En cambio, es usada por el ambiente de simulación de SimPy. El ambiente al ejecutarse, recibe este parámetro como "until", es decir, que se ejecuta hasta que el reloj interno alcance este valor.

**b. Variables del modelado**

El uso de Simlib, permite facilitar la declaración de variables explícitas, y la definición de los métodos de la simulación. Esto es, porque la propia librería tiene predefinidos múltiples variables y métodos en cada simulación, realizada con esta. Por tanto, muchas de las variables específicas de cada agente, evento o proceso, son manejadas internamente por Simlib, y no tienen que ser escritas en el código. En la siguiente tabla, mostramos las variables usadas de forma explícita en la simulación.

| Nombre          | Tipo  | Uso                                                                 |
|-----------------|-------|---------------------------------------------------------------------|
| sim_time        | Float | Reloj interno del simulador                                         |
| next_event_type | Int   | Identificador del tipo de evento que se debe planear a continuación |
| maxatr          | Int   | Número máximo de atributos por lista                                |
| maxlist         | Int   | Número máximo de listas                                             |

Mantenemos las llamadas al reloj interno, y al identificador de tipo de evento. El reloj sigue siendo necesario manejarlo, para especificar el manejo de eventos de esta simulación en particular. Por ejemplo, en este programa que requiere indicar de entrada, el tipo de distribuciones (exponencial o constante) con el que se van a programar las llegadas y servicios de los elementos, es necesario especificar cómo se van a calcular, usando sim_time como referente. El identificador de tipo de evento es necesario para dar forma a la simulación particular. Por ejemplo, es necesario identificar el orden para crear una simulación de dos servicios secuenciales, en vez de paralelos.
Finalmente, maxatr y maxlist deben ser definidos en cualquier simulación que utilice simlib. La variable maxatr define el número máximo de atributos de cada lista simlib. Por la manera en la que esta escrito Simlib internamente, el mínimo valor posible de maxatr es 4. Por otra parte, maxlist indica el máximo de listas que se pueden crear en una simulación. El mínimo valor que se puede usar es 25, porque la lista 25 siempre va a ser la lista de eventos.


**c. Descripción del proceso y tipo de proceso**

| Nombre     | Parámetros | Uso                                                                                                           |
|------------|------|---------------------------------------------------------------------------------------------------------------|
| customer_generator    | Ninguno    | Crea un cliente (con id), registra su llegada al sistema, ejecuta customer_flow y programa al siguiente cliente|
| customer_flow | customer_id, arrival_time   | Registra el ingreso a la fila 1, demora respectiva, tiempo de servicio 1, y el mismo proceso del servicio 2 |
| monitor_queues | Ninguno   | Mide la cantidad de clientes haciendo fila en el tiempo                 |
| monitor_servers | Ninguno    | Monitorea el uso de cada servidor en el tiempo                |

SimPy, a diferencia de Simlib, es una librería de modelado por procesos. Los procesos, son más afines a los hilos de un sistema operativo. Mientras que el sistema por eventos, define los eventos que se deben detectar, y las acciones a realizar cuando ocurre un evento, el modelado por procesos define las acciones que debe realizar cada agente, y construye el hilo que debe seguir cada agente en el modelo. Para este ejemplo, definimos 4 procesos. El proceso principal es "customer_flow", que indica el camino que realiza un agente (cliente) en el sistema. Este proceso define cuando entra a la fila 1, la espera al servicio 1, la ejecución del servicio 1, la entrada a la fila 2, la espera del servicio 2, la ejecución del servicio 2, y la salida del sistema. Este proceso recibe un "cliente", el cual es generado por el proceso "customer_generator" de acuerdo a los tiempos de llegada que reciba la simulación. Una vez se crea un cliente, este realiza el proceso de flujo del cliente. El proceso de "customer_flow" también registra los tiempos de espera en fila promedio que se usarán para la evaluación de métricas. Los otros dos modelos se encargan de monitorear la cantidad de clientes haciendo fila, y el uso de servidores respectivamente, lo que también dará un resultado para la evaluación de métricas.

**d. Listas y sus atributos**

Nombre     | Llave                | Descripción     |                                                                              
------------|-------------------------------|----------------|
 queue1_delays     |                 | Mide la diferencia entre el tiempo actual, y la llegada del cliente a la cola 1 |
 queue2_delays    |                    | Mide la diferencia entre el tiempo actual, y la llegada del cliente a la cola 2                |
 queue1_lengths    |                  | Registra la cantidad de clientes haciendo fila para el servicio 1 cada 6 segundos |
 queue2_lengths    |                   | Registra la cantidad de clientes haciendo fila para el servicio 2 cada 6 segundos                |
 server1_busy |  | Registra cada 6 segundos si el servidor 1 está ocupado |
 server2_busy |  | Registra cada 6 segundos si el servidor 2 está ocupado |
 queue1_entry_times | customer_id | Registra la entrada a la cola 1 para cada cliente |
 queue2_entry_times | customer_id | Registra la entrada a la cola 2 para cada cliente |
 
Las listas en SimPy pueden ser arreglos convencionales, o pueden ser diccionarios, dependiendo de si el valor almacenado es específico de un cliente, o no. En el caso de queue1_entry_times y queueu2_entry_times, es necesario asociarlo a cada cliente, para poder monitorear la demora en las filas a lo largo del flujo de cada cliente en específico. Si no estuvieran marcadas, existe la posibilidad de que se registren entradas a la cola para clientes distintos, porque el tiempo de servicio sea diferente. En cambio, las otras listas no deben ser diccionarios, porque todas las mediciones son medidores de rendimiento directos en un punto determinado, y no dependen del seguimiento del cliente para su cálculo.

**e. Contadores y/o acumuladores**

| Variable                | Tipo            | Descripción                                           | 
|-------------------------|-----------------|-------------------------------------------------------|
| num_arrivals      | Contador entero | Clientes que no han sido atendidos de forma inmediata |
| num_completed| Contador entero | Clientes que han completado el servicio 2             |

El uso de contadores no requiere múltiples modificaciones respecto al programa original en Simlib, por lo que se mantiene el sistema original.

**f. Medidas de desempeño**

| Variable                 Descripción                                           | 
|--------------------------------------------------------------------------------|
| Demora promedio en cola| Tiempo promedio de espera en cualquiera de las dos colas            |
| Demora máxima en cola      | Tiempo máximo de espera registrado en cola |
| Demora mínima en cola     | Tiempo mínimo de espera registrado en cola |
| Longitud promedio de cola 1    | Contador entero | Clientes que no han sido atendidos de forma inmediata |
| Longitud máxima de cola 1      | Contador entero | Clientes que no han sido atendidos de forma inmediata |
| Longitud mínima de cola 1      | Contador entero | Clientes que no han sido atendidos de forma inmediata |
| Longitud promedio de cola 2    | Contador entero | Clientes que no han sido atendidos de forma inmediata |
| Longitud máxima de cola 2      | Contador entero | Clientes que no han sido atendidos de forma inmediata |
| Longitud mínima de cola 2      | Contador entero | Clientes que no han sido atendidos de forma inmediata |
| Uso promedio del servidor 1    | Contador entero | Clientes que no han sido atendidos de forma inmediata |
| Uso máximo del servidor 1      | Contador entero | Clientes que no han sido atendidos de forma inmediata |
| Uso mínimo del servidor 1     | Contador entero | Clientes que no han sido atendidos de forma inmediata |
| Uso promedio del servidor 2   | Contador entero | Clientes que no han sido atendidos de forma inmediata |
| Uso máximo del servidor 2     | Contador entero | Clientes que no han sido atendidos de forma inmediata |
| Uso mínimo del servidor 2     | Contador entero | Clientes que no han sido atendidos de forma inmediata |

Se usaron los mismos marcadores de rendimiento de Simlib, para poder comparar fielmente los resultados obtenidos en ambos modelos.

**g. Subprogramas y propósito**

Igualmente, para este problema se decidieron no realizar subprogramas. Esto, por la misma razón que no se usaron en Simpy. Para la ejecución de la solución en este Notebook, se consideró más práctico hacer el programa flexible a distintos parámetros, y producir una salida distinta según los parámetros, en vez de ejecutar un modelo aparte con modificaciones mínimas.

### 2. Elaborar el diagrama de flujo del programa principal y de cada subprograma que conforma el modelo

### 3. Desarrollar el simulador en lenguaje de alto nivel

In [8]:
%%writefile parameters_9.txt
10 9 10000 1 1 1

Writing parameters_10.txt


In [ ]:
!pip install simpy

import simpy
import random
import statistics
from collections import defaultdict

class SimuladorSecuencial:
    def __init__(self, env, mean_interarrival, mean_service, 
                 servers_level1, servers_level2, distribution_type):
        self.env = env
        self.mean_interarrival = mean_interarrival
        self.mean_service = mean_service
        self.distribution_type = distribution_type
        self.server1 = simpy.Resource(env, capacity=servers_level1)
        self.server2 = simpy.Resource(env, capacity=servers_level2)
        self.queue1_delays = []
        self.queue2_delays = []
        self.system_times = []
        self.queue1_lengths = []
        self.queue2_lengths = []
        self.server1_busy = []
        self.server2_busy = []
        self.num_completed = 0
        self.num_arrivals = 0
        self.queue1_entry_times = {}
        self.queue2_entry_times = {}
        self.env.process(self.customer_generator())
        self.env.process(self.monitor_queues())
        self.env.process(self.monitor_servers())
    
    def get_interarrival_time(self):
        if self.distribution_type in [1, 3]:
            return random.expovariate(1.0 / self.mean_interarrival)
        else:
            return self.mean_interarrival
    
    def get_service_time(self):
        if self.distribution_type in [1, 2]:
            return random.expovariate(1.0 / self.mean_service)
        else:
            return self.mean_service
    
    def customer_generator(self):
        while True:
            # Schedule next arrival first (like in original arrive function)
            yield self.env.timeout(self.get_interarrival_time())
            
            # Create new customer (equivalent to transfer[1] = sim_time in original)
            arrival_time = self.env.now
            self.num_arrivals += 1
            customer_id = self.num_arrivals
            
            # Process customer through the system
            self.env.process(self.customer_flow(customer_id, arrival_time))
    
    def customer_flow(self, customer_id, arrival_time):
        """Handles the complete flow through both queues and servers"""
        # Level 1 processing (equivalent to original arrive and depart1 logic)
        
        # Record queue1 entry time (transfer[2] in original)
        queue1_entry = self.env.now
        self.queue1_entry_times[customer_id] = queue1_entry
        
        # Request server1 (this handles both queueing and service)
        with self.server1.request() as req:
            # This is where the customer would wait in queue1 if servers are busy
            yield req
            
            # Customer got server1 - record delay
            queue1_delay = self.env.now - queue1_entry
            self.queue1_delays.append(queue1_delay)
            del self.queue1_entry_times[customer_id]
            
            # Process at server1 (equivalent to EVENT_DEPARTURE1)
            yield self.env.timeout(self.get_service_time())
        
        # Level 2 processing (equivalent to original depart1 and depart2 logic)
        
        # Record queue2 entry time (transfer[4] in original)
        queue2_entry = self.env.now
        self.queue2_entry_times[customer_id] = queue2_entry
        
        # Request server2
        with self.server2.request() as req:
            # Wait in queue2 if servers are busy
            yield req
            
            # Customer got server2 - record delay
            queue2_delay = self.env.now - queue2_entry
            self.queue2_delays.append(queue2_delay)
            del self.queue2_entry_times[customer_id]
            
            # Process at server2 (equivalent to EVENT_DEPARTURE2)
            yield self.env.timeout(self.get_service_time())
        
        # Customer completed both services
        system_time = self.env.now - arrival_time
        self.system_times.append(system_time)
        self.num_completed += 1
    
    def monitor_queues(self):
        """Tracks queue lengths over time (equivalent to original filest)"""
        while True:
            # Current customers waiting in queue1 (not being served)
            q1_len = len(self.server1.queue)
            self.queue1_lengths.append(q1_len)
            
            # Current customers waiting in queue2 (not being served)
            q2_len = len(self.server2.queue)
            self.queue2_lengths.append(q2_len)
            
            yield self.env.timeout(0.1)  # Sample every 0.1 time units
    
    def monitor_servers(self):
        """Tracks server utilization over time"""
        while True:
            # Record number of busy servers (equivalent to list_size[LIST_SERVER1])
            self.server1_busy.append(len(self.server1.users))
            self.server2_busy.append(len(self.server2.users))
            yield self.env.timeout(0.1)  # Sample every 0.1 time units
    
    def generate_report(self):
        """Produces output exactly matching the original report format"""
        def safe_mean(data):
            return statistics.mean(data) if data else 0.0
        
        def safe_max(data):
            return max(data) if data else 0.0
        
        def safe_min(data):
            return min(data) if data else 0.0
        
        report = "Multi-server queueing system using SimPy\n\n"
        report += f"Mean interarrival time{11*' '}{self.mean_interarrival:.3f} minutes\n\n"
        report += f"Mean service time{16*' '}{self.mean_service:.3f} minutes\n\n"
        report += f"Total simulation time{12*' '}{self.env.now:.3f} minutes\n\n"
        report += f"Distribution type: {self.distribution_type}\n"
        
        # Distribution description matching original
        dist_map = {
            1: "Interarrival: Exponential, Service: Exponential",
            2: "Interarrival: Constant, Service: Exponential",
            3: "Interarrival: Exponential, Service: Constant",
            4: "Interarrival: Constant, Service: Constant"
        }
        report += dist_map.get(self.distribution_type, "Invalid distribution type") + "\n"
        report += f"Number of servers at level 1: {self.server1.capacity}\n"
        report += f"Number of servers at level 2: {self.server2.capacity}\n\n"
        
        # Queue 1 delays
        report += "\nDelays in Queue 1, in minutes:\n"
        report += f"  Average: {safe_mean(self.queue1_delays):.3f}\n"
        report += f"  Maximum: {safe_max(self.queue1_delays):.3f}\n"
        report += f"  Minimum: {safe_min(self.queue1_delays):.3f}\n"
        report += f"  Number measured: {len(self.queue1_delays)}\n"
        
        # Queue 2 delays
        report += "\nDelays in Queue 2, in minutes:\n"
        report += f"  Average: {safe_mean(self.queue2_delays):.3f}\n"
        report += f"  Maximum: {safe_max(self.queue2_delays):.3f}\n"
        report += f"  Minimum: {safe_min(self.queue2_delays):.3f}\n"
        report += f"  Number measured: {len(self.queue2_delays)}\n"
        
        # System times
        report += "\nTotal System Time (arrival to final departure), in minutes:\n"
        report += f"  Average: {safe_mean(self.system_times):.3f}\n"
        report += f"  Maximum: {safe_max(self.system_times):.3f}\n"
        report += f"  Minimum: {safe_min(self.system_times):.3f}\n"
        report += f"  Number measured: {len(self.system_times)}\n"
        
        # Queue lengths (average)
        report += "\nQueue lengths:\n"
        report += f"  Average queue 1 length: {safe_mean(self.queue1_lengths):.3f}\n"
        report += f"  Average queue 2 length: {safe_mean(self.queue2_lengths):.3f}\n"
        
        # Server utilization (percentage)
        server1_util = sum(self.server1_busy) / len(self.server1_busy) / self.server1.capacity * 100
        server2_util = sum(self.server2_busy) / len(self.server2_busy) / self.server2.capacity * 100
        report += "\nServer utilization:\n"
        report += f"  Server 1 utilization: {server1_util:.1f}%\n"
        report += f"  Server 2 utilization: {server2_util:.1f}%\n"
        
        report += f"\nTime simulation ended:{12*' '}{self.env.now:.3f} minutes\n"
        report += f"Number of customers completed both services:{8*' '}{self.num_completed}\n"
        
        return report

def read_parameters(filename):
    """Reads parameters from file in the same format as the original"""
    with open(filename, 'r') as f:
        # Read all values from the first line
        values = list(map(float, f.readline().strip().split()))
        
        # Convert to appropriate types
        mean_interarrival = values[0]
        mean_service = values[1]
        total_simulation_time = int(values[2])
        distribution_type = int(values[3])
        servers_level1 = int(values[4])
        servers_level2 = int(values[5])
        
        return (mean_interarrival, mean_service, total_simulation_time,
                distribution_type, servers_level1, servers_level2)

def run_simulation(input_file, output_prefix):
    """Main function to run the simulation"""
    try:
        # Read parameters from file
        params = read_parameters(input_file)
        mean_interarrival, mean_service, total_simulation_time, \
        distribution_type, servers_level1, servers_level2 = params
        
        # Run simulation
        env = simpy.Environment()
        system = SimuladorSecuencial(env, mean_interarrival, mean_service,
                                   servers_level1, servers_level2, distribution_type)
        env.run(until=total_simulation_time)
        
        # Generate and save report
        report = system.generate_report()
        output_filename = f"{output_prefix}_{distribution_type}.txt"
        
        with open(output_filename, 'w') as f:
            f.write(report)
        
        print(f"Simulation complete. Report saved to {output_filename}")
        return True
    
    except FileNotFoundError:
        print(f"Error: Cannot open input file {input_file}")
        return False
    except Exception as e:
        print(f"Error running simulation: {str(e)}")
        return False

if __name__ == "__main__":
    # Run with the same file names as original
    input_file = "parameters_9.txt"
    output_prefix = "report_9"
    
    if not run_simulation(input_file, output_prefix):
        # If parameters_9.txt not found, create a sample one and try again
        print("Creating sample parameters file and running simulation...")
        with open(input_file, 'w') as f:
            f.write("5.0 4.0 1000 1 2 3\n")  # Default parameters
        
        run_simulation(input_file, output_prefix)

### 4. Analizar resultados

In [12]:
with open("report_9_1.txt", "r") as file:
    print(file.read())


FileNotFoundError: [Errno 2] No such file or directory: '/content/report_10.txt'

**a.	Corrida 1: tiempo entre llegada y de servicio exponenciales**

Multi-server queueing system using simlib

Mean interarrival time     10.000 minutes

Mean service time           9.000 minutes

Total simulation time       10000 minutes

Distribution type: 1
Interarrival: Exponential, Service: Exponential
Number of servers at level 1: 1
Number of servers at level 2: 1


Delays in queue, in minutes:

 sampst                         Number
variable                          of
 number       Average           values          Maximum          Minimum
________________________________________________________________________

    1         121.123          1014.00          349.119          0.00000 
________________________________________________________________________



Queue length and server utilization:

  File         Time
 number       average          Maximum          Minimum
_______________________________________________________

    1         12.2813          37.0000          0.00000 

    2        0.925342          1.00000          0.00000 
_______________________________________________________



  File         Time
 number       average          Maximum          Minimum
_______________________________________________________

    3         5.06968          28.0000          0.00000 

    4        0.862937          1.00000          0.00000 
_______________________________________________________



Time simulation ended:   10000.495 minutes
Number of customers completed both services:    1006
Number of servers at level 1: 1
Number of servers at level 2: 1


**b.	Corrida 2: Tiempo entre llegada constantes y tiempos de servicio exponenciales**

In [8]:
%%writefile parameters_9.txt
10 9 10000 2 1 1

Writing parameters_10.txt


In [11]:
!./simulation_9

In [12]:
with open("report_9_2.txt", "r") as file:
    print(file.read())


FileNotFoundError: [Errno 2] No such file or directory: '/content/report_10.txt'

Multi-server queueing system using simlib

Mean interarrival time     10.000 minutes

Mean service time           9.000 minutes

Total simulation time       10000 minutes

Distribution type: 2
Interarrival: Constant, Service: Exponential
Number of servers at level 1: 1
Number of servers at level 2: 1


Delays in queue, in minutes:

 sampst                         Number
variable                          of
 number       Average           values          Maximum          Minimum
________________________________________________________________________

    1         36.6710          998.000          148.308          0.00000 
________________________________________________________________________



Queue length and server utilization:

  File         Time
 number       average          Maximum          Minimum
_______________________________________________________

    1         3.66076          15.0000          0.00000 

    2        0.894527          1.00000          0.00000 
_______________________________________________________



  File         Time
 number       average          Maximum          Minimum
_______________________________________________________

    3         3.54243          25.0000          0.00000 

    4        0.864306          1.00000          0.00000 
_______________________________________________________



Time simulation ended:   10000.000 minutes
Number of customers completed both services:     993
Number of servers at level 1: 1
Number of servers at level 2: 1


**c.	Corrida 3: Tiempo entre llegadas exponenciales y tiempo de servicio constantes**

In [8]:
%%writefile parameters_9.txt
10 9 10000 3 1 1

Writing parameters_10.txt


In [11]:
!./simulation_9

In [12]:
with open("report_9_3.txt", "r") as file:
    print(file.read())


FileNotFoundError: [Errno 2] No such file or directory: '/content/report_10.txt'

Multi-server queueing system using simlib

Mean interarrival time     10.000 minutes

Mean service time           9.000 minutes

Total simulation time       10000 minutes

Distribution type: 3
Interarrival: Exponential, Service: Constant
Number of servers at level 1: 1
Number of servers at level 2: 1


Delays in queue, in minutes:

 sampst                         Number
variable                          of
 number       Average           values          Maximum          Minimum
________________________________________________________________________

    1         47.3141          1015.00          208.489          0.00000 
________________________________________________________________________



Queue length and server utilization:

  File         Time
 number       average          Maximum          Minimum
_______________________________________________________

    1         4.79695          24.0000          0.00000 

    2        0.911567          1.00000          0.00000 
_______________________________________________________



  File         Time
 number       average          Maximum          Minimum
_______________________________________________________

    3         0.00000          1.00000          0.00000 

    4        0.911567          1.00000          0.00000 
_______________________________________________________



Time simulation ended:   10011.330 minutes
Number of customers completed both services:    1014
Number of servers at level 1: 1
Number of servers at level 2: 1


**d.	Corrida 4: Tiempo entre llegadas constantes y tiempos de servicio constantes**

In [8]:
%%writefile parameters_9.txt
10 9 10000 4 1 1

Writing parameters_10.txt


In [11]:
!./simulation_9

In [12]:
with open("report_9_4.txt", "r") as file:
    print(file.read())


FileNotFoundError: [Errno 2] No such file or directory: '/content/report_10.txt'

Multi-server queueing system using simlib

Mean interarrival time     10.000 minutes

Mean service time           9.000 minutes

Total simulation time       10000 minutes

Distribution type: 4
Interarrival: Constant, Service: Constant
Number of servers at level 1: 1
Number of servers at level 2: 1


Delays in queue, in minutes:

 sampst                         Number
variable                          of
 number       Average           values          Maximum          Minimum
________________________________________________________________________

    1         0.00000          1000.00          0.00000          0.00000 
________________________________________________________________________



Queue length and server utilization:

  File         Time
 number       average          Maximum          Minimum
_______________________________________________________

    1         0.00000     -1.00000E+30      1.00000E+30 

    2        0.899100          1.00000          0.00000 
_______________________________________________________



  File         Time
 number       average          Maximum          Minimum
_______________________________________________________

    3         0.00000     -1.00000E+30      1.00000E+30 

    4        0.898300          1.00000          0.00000 
_______________________________________________________



Time simulation ended:   10000.000 minutes
Number of customers completed both services:     998
Number of servers at level 1: 1
Number of servers at level 2: 1


**e. Análisis de resultados**

En esta sección, acabamos de mostrar cómo se realizan reportes de rendimiento para las cuatro corridas requeridas para el ejercicio. En la siguiente tabla, comparamos las corridas realizadas durante el desarrollo de este taller. Se pueden confirmar estos valores en el repositorio oficial del taller:
| Métrica            | Exponencial/Exponencial | Constante/Exponencial | Exponencial/Constante | Constante/Constante |
|--------------------|-------------------------|-----------------------|-----------------------|---------------------|
| Demora total       | 145.865 (4.5-341.9)     | 124.746 (25.1-223.2)  | 85.424 (18-237.2)     | 18 (18-18)          |
| Demora cola 1      | 75.27 (0-306.2)         | 39.169 (0-169.4)      | 67.317 (0-219.2)      | 0                   |
| Demora cola 2      | 51.864 (0-256)          | 66.972 (0-205)        | 0                     | 0                   |
| Longitud fila 1    | 7.551                   | 3.914                 | 6.943                 | 0                   |
| Longitud fila 2    | 5.203                   | 6.637                 | 0                     | 0                   |
| Uso servidor 1     | 88.4%                   | 91.5%                 | 92.7%                 | 89.9%               |
| Uso servidor 2     | 88.8%                   | 93%                   | 92.6%                 | 89.8%               |
| Clientes atendidos | 988                     | 990                   | 1029                  | 998                 |

Comparando con los resultados obtenidos con Simlib, observamos que en general, son bastante parecidos. El resultado de la Corrida 4, de tiempos constantes, es exactamente idéntico, mientras que las otras corridas tienen leves variaciones. Esto puede ser resultado de la variabilidad dada por el manejo de variables aleatorias en ambos programas. No obstante, al observar las mismas tendencias dependiendo del manejo de las variables de cada corrida, se puede confirmar que la simulación por procesos hecha en SimPy, es equivalente a la simulación por eventos en Simlib.

### 5. Plantear modificaciones

Para el proceso de modificaciones, vamos a realizar experimentos únicamente en parámetros que se presume, pueden ser modificados por el sistema. Por ejemplo, el tiempo de llegada de clientes (y su distribución), no es dependiente del funcionamiento de su sistema, y se presume que no puede ser modificado por acciones reales del sistema. Por tanto, nos vamos a enfocar en dos variables, el número de servidores, y el tiempo promedio de servicio. Comparar estas dos variables, nos permite asesorar, por ejemplo, si es más eficiente poner una nueva máquina (en el caso de que sea una planta de ensamblaje, equivalente a duplicar servidores), o si es más eficiente reemplazar la máquina existente por una más rápida (disminuyendo el tiempo de servicio). También es importante determinar cuál de las dos máquinas vale más la pena duplicar o reemplazar. A continuación, mostramos las corridas para el modo 1, con las modificaciones pertinentes: